In [2]:
!pip install torchinfo

In [3]:
import os

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchinfo import summary
from torchvision import datasets, models, transforms

# Important! Don't change this
torch.backends.cudnn.deterministic = True

We'll be making large networks, so we'll get the GPU if it's available.

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using {device} device.")

**Load and Explore the Data**

!ls -l potato_dataset/

In [1]:
!ls -l potato_dataset/

ls: cannot access 'potato_dataset/': No such file or directory


**Task 2.7.1: Make a list of the names of the classes we'll be working with.**

In [ ]:
train_dir = os.path.join('potato_dataset','train')
classes = os.listdir(train_dir)

print(classes)
print(f"Number of classes: {len(classes)}")








Now we're ready to start preparing our dataset. For now, let's use the following set of transformations:

Convert any grayscale images to RGB format with a custom class
Resize the image, so that they're all the same size (we chose
 x
)
Convert the image to a Tensor of pixel values

**Task 2.7.2:** **Create the transformation pipeline with steps listed above by using `transforms.Compose` from the `torchvision` package**.

In [ ]:
height = 224
width = 224


class ConvertToRGB:
    def __call__(self, img):
        if img.mode != "RGB":
            img = img.convert("RGB")
        return img


transform = transforms.Compose([
     ConvertToRGB(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
]
)

print(transform)

**Task 2.7.3: Make a dataset using ImageFolder from datasets and make sure to apply the transform transformation pipeline.**

In [ ]:
dataset = datasets.ImageFolder(root=train_dir, transform=transform)

print("Length of dataset:", len(dataset))

Notice how small this training data is, we have just 421 images in total.

** Task 2.7.4: Use this dataset to create a DataLoader with batch size 32.**

In [ ]:
batch_size = 32
dataset_loader = DataLoader(dataset, batch_size=batch_size)
batch_shape = next(iter(dataset_loader))[0].shape
print("Getting batches of shape:", batch_shape)

Now we can think about normalizing our dataset, so each color channel has a mean of
 and a standard deviation of
. In order to do this, we first need to get the mean and standard deviation of each channel in this dataset.

The get_mean_std function we used in the lessons is included in the training.py file for this assignment. Let's import it.

**Task 2.7.5: Use function get_mean_std described above and calculate the mean and standard deviation of each channel in this dataset.**

In [ ]:
mean, std = get_mean_std(dataset_loader)

print(f"Mean: {mean}")
print(f"Standard deviation: {std}")

Great! Now we can improve our original transformation pipeline. We'll take the previous one, and add normalization of the channels according to the mean and standard deviation you computed in the previous task. Our transformations should now look like:

Convert any grayscale images to RGB format with a custom class
Resize the image, so that they're all the same size (we chose
 x
)
Convert the image to a Tensor of pixel values
Normalize the data using transforms.Normalize

**Task 2.7.6: Build this new transformation pipeline.**

In [ ]:
transform_norm = transforms.Compose(
    [
        ConvertToRGB(),
        transforms.Resize((width, height)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ]
)

print(transform_norm)

**Task 2.7.7: Create a normalized data set, using the transformation pipeline from the previous task. Also split the normalized data set into a training set and a validation set. 80% of the data should be in the training set, and 20% in the validation set.**

In [ ]:
# Important! don't change this.
g = torch.Generator()
g.manual_seed(42)

norm_dataset = datasets.ImageFolder(root=train_dir , transform =transform_norm)

# Important, DON'T change the `generator=g` parameter
train_dataset, val_dataset = random_split(norm_dataset,[0.8,0.2], generator=g)

print("Length of dataset:", len(norm_dataset))
print("Training data set size:", len(train_dataset))
print("Validation data set size:", len(val_dataset))

Now let's create DataLoader objects. We'll use a batch size of 32 and create one DataLoader for training and another for validation data.

**Task 2.7.8: Create the training loader (with shuffling on) and the validation loader (with shuffling off).**

In [ ]:
# Important! don't change this.
g = torch.Generator()
g.manual_seed(42)

batch_size = 32

# Important! Don't change the `generator=g` parameter
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True, generator=g)
val_loader = DataLoader(val_dataset,batch_size=batch_size)

print(type(train_loader))
print(type(val_loader))

**Task 2.7.9: Fill in the missing code below to define a neural network with the following layers:**

2D convolution with sixteen

 kernels

Max pooling with

 kernels (and a stride of
)

ReLU activation

2D convolution with thirty-two

 kernels

Max pooling with

 kernels

ReLU activation

2D convolution with sixty-four

 kernels

Max pooling with


 kernels

ReLU activation

Flattening

Drop-out

Linear layer with

 outputs

ReLU activation


Drop-out

Linear output layer with the correct number of outputs

In [ ]:
# Important! Don't change this
torch.manual_seed(42)
torch.cuda.manual_seed(42)

model = torch.nn.Sequential()

conv1_n_kernels = 16
conv1 = torch.nn.Conv2d(
    in_channels=3, out_channels=conv1_n_kernels, kernel_size=(3, 3), padding=1
)
max_pool_size = 4
max_pool1 = torch.nn.MaxPool2d(max_pool_size)
model.append(conv1)
model.append(torch.nn.ReLU())
model.append(max_pool1)

conv2_n_kernels = 32
conv2 = torch.nn.Conv2d(
    in_channels=16, out_channels=conv2_n_kernels, kernel_size=(3, 3), padding=1
)
max_pool2 = torch.nn.MaxPool2d(max_pool_size)
model.append(conv2)
model.append(torch.nn.ReLU())
model.append(max_pool2)

conv3_n_kernels = 64
conv3 = torch.nn.Conv2d(32, conv3_n_kernels, 3, padding=1)
max_pool3 = torch.nn.MaxPool2d(max_pool_size)
model.append(conv3)
model.append(torch.nn.ReLU())
model.append(max_pool3)

model.append(torch.nn.Flatten())
model.append(torch.nn.Dropout())

linear1 = torch.nn.Linear(in_features=576, out_features=500)
model.append(linear1)
model.append(torch.nn.ReLU())
model.append(torch.nn.Dropout())

n_classes = 3
output_layer = torch.nn.Linear(500, n_classes)
model.append(output_layer)

summary(model, input_size=(batch_size, 3, height, width))

**Task 2.7.10: Define cross-entropy as the loss function and set Adam optimizer to be the optimizer. You can use the default learning settings on Adam (but it needs to know what parameters to use, so don't forget model.parameters()). Make sure to also send the model to the GPU device.**

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Make sure to send the model to the GPU

print(loss_fn)
print("----------------------")
print(optimizer)
print("----------------------")
print(next(model.parameters()).device)

For training the model, we can first use the functions we have in the training.py file. This model should train fairly quickly with such a small dataset, so we should leave the use_train_accuracy at default (slower training, better information).

**Task 2.7.11: Use the train function from training.py to train the model for 15 epochs.**

In [ ]:
# Import the train function from `training.py`
from training import train
# Train the model for 15 epochs
epochs =15

train_losses, val_losses, train_accuracies, val_accuracies = train(model,optimizer,loss_fn, train_loader,val_loader,epochs,device=device)

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Loss over epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label="Training Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.title("Accuracy over epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

If everything was set up correctly, you'll notice that some learning happened. Well done! The results are not too bad, but we can do better. We could invest energy in preventing overfitting and improving the model, and training longer. But the biggest problem is that our dataset is small. A problem like this calls for Transfer Learning!

**Transfer Learning**

With so little data, we would be well served by getting a model that's been trained on a larger dataset. The restnet50 model we used in the lessons does well with a variety of images. Let's fetch that model and use it for Transfer Learning.

**Task 2.7.12: Set up the resnet50 model. We'll alter it to suit our task in subsequent steps.**

In [ ]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

print(model)

**Task 2.7.13: Fix the parameters of the model, so that they won't be updated when we train the model on our data.**

In [ ]:
# Freeze the models weights
for params in model.parameters():
    params.requires_grad=False
print(model)

Now we want to change the output layer of the model (layer model.fc). You'll first need to compute the number of features going into the last layer of the original model. Then you can change the last layer into:


a dense layer with 256 neurons

followed by ReLU activation

then add p=0.5 of Dropout

followed by the output layer with 3 outputs

In [ ]:
# Important! Don't change this
torch.manual_seed(42)
torch.cuda.manual_seed(42)

in_features = model.fc.in_features

modified_last_layer = nn.Sequential()

dense_layer = nn.Linear(in_features,256)
modified_last_layer.append(dense_layer)

relu = nn.ReLU()
modified_last_layer.append(relu)

modified_last_layer.append(nn.Dropout(p=0.5))

output_layer = nn.Linear(256,3)
modified_last_layer.append(output_layer)

# Assign `modified_last_layer` to `model.fc`
model.fc = modified_last_layer

print(model)

In [ ]:
# Important! don't change this.
g = torch.Generator()
g.manual_seed(42)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=g)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Task 2.7.15: Define the loss function, create an Adam optimizer and send model to GPU device. Now we are ready to train the model for
 epochs. **bold text**

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
# Place the model on device
model.to(device)
# Train the model for 10 epochs
epochs = 10
train_losses, val_losses, train_accuracies, val_accuracies = train(
    model, optimizer, loss_fn, train_loader, val_loader, epochs, device=device
)

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Loss over epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label="Training Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.title("Accuracy over epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

**Task 2.7.16: Create a dataset for the test data. The data is located in the potato_dataset/test directory. Then make a data loader from this data set and make sure to not shuffle this data! We'll use a smaller batch size here (batch_size = 10) because the test dataset contains only 35 images.**

In [ ]:
from training import predict

test_dir =os.path.join ('potato_dataset','test')
test_dataset = datasets.ImageFolder(
    root=test_dir, transform=transform_norm
)


batch_size = 10


test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)

print("Number of test images:", len(test_dataset))

We want to know the right category (healthy, early blight, or late blight) for each test image. We're not asked for the probabilities, so we'll need to get the actual predicted categories. Someone else looking at this won't know what our class numbers mean, so there is also code that turns the numbers into names

**Task 2.7.17: Make a prediction for each of the test images.**

In [ ]:
# Predict the probabilities for each test image
test_probabilities = predict(model,test_loader , device)

# Get the index associated with the largest probability for each test image
test_predictions = torch.argmax(test_probabilities,dim=1)
# Converts the class index to the class name for each test image.
test_classes = [train_dataset.dataset.classes[i] for i in test_predictions]

print("Number of predictions:", test_predictions.shape)
print("Predictions (class index):", test_predictions.tolist())
print()
print("Predictions (class name):", test_classes)

**Training with Callbacks**

The learning curve we obtained after training this model suggests that we've likely fit long enough. But a couple more epochs could help. The best way would be to just train for very long, but stop when training stops improving. This requires us using Callbacks.

In lesson 025-callbacks, we modified the training loop to include a few callbacks. That code is in this assignment's training.py, but it's now named train_callback. Let's import it.

In [ ]:
train_callbacks?

**Training with Callbacks**

The learning curve we obtained after training this model suggests that we've likely fit long enough. But a couple more epochs could help. The best way would be to just train for very long, but stop when training stops improving. This requires us using Callbacks.

In lesson 025-callbacks, we modified the training loop to include a few callbacks. That code is in this assignment's training.py, but it's now named train_callback. Let's import it.

In [ ]:
from training import train_callbacks


train_callbacks?

**Task 2.7.18: Continue training the model for another 50 epochs and make sure to include Early Stopping and Checkpointing. You'll need to define the checkpointing path, use "LR_model.pth". For early_stopping_function, there's an early_stopping function in the training.py. Import that and use it.**

In [ ]:
# Import the early_stopping function
from training import early_stopping
epochs_to_train = 50
checkpoint_path = "LR.model.pth"
early_stopping_function = early_stopping

train_results = train_callbacks(
    model,
    optimizer,
    loss_fn,
    train_loader,
    val_loader,
    epochs=epochs_to_train,
    device=device,
    checkpoint_path=checkpoint_path,
    early_stopping=early_stopping_function,
)

(
    learning_rates,
    train_losses,
    valid_losses,
    train_accuracies,
    valid_accuracies,
    epochs,
) = train_results

With early stopping, the model trained a few epochs past the best model. But the checkpointing saved that best model. Let's load it.

**Task 2.7.19: Load the saved best model from the checkpointing.**

In [ ]:
# Load the model with `torch.load`
checkpoint = torch.load("LR_model.pth")
# Load model state dictionaries
model.load_state_dict(checkpoint["model_state_dict"])
print(model)

**Task 2.7.20: Use the loaded best model to make predictions on each test image.**

In [ ]:
# Predict the probabilities for each test image
test_probabilities = predict(model, test_loader, device)

# Get the index associated with the largest probability for each test image
test_predictions = torch.argmax(test_probabilities, dim=1)

# Converts the class index to the class name for each test image.
test_classes = [train_dataset.dataset.classes[i] for i in test_predictions]

print("Number of predictions:", test_predictions.shape)
print("Predictions (class index):", test_predictions.tolist())
print()
print("Predictions (class name):", test_classes)
